In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

In [2]:
# 1. Define the Graph State
# This is what will be passed around between nodes
class GraphState(TypedDict):
    number: Annotated[int, operator.add] # The input number
    action_taken: str # To record what action was taken

In [3]:
# 2. Define the Nodes
# Each node is a Python function that takes the current state and returns a dictionary to update the state.

def check_even_odd(state: GraphState) -> dict:
    """
    Checks if the number is even or odd.
    Does not modify the state directly, just determines the next step.
    """
    number = state["number"]
    print(f"Node: check_even_odd - Checking number: {number}")
    if number % 2 == 0:
        return {"next_node": "process_even"}
    else:
        return {"next_node": "process_odd"}

def process_even(state: GraphState) -> dict:
    """
    Action for an even number.
    """
    number = state["number"]
    print(f"Node: process_even - Processing even number: {number}")
    new_action = f"Processed {number} as EVEN."
    return {"action_taken": new_action}

def process_odd(state: GraphState) -> dict:
    """
    Action for an odd number.
    """
    number = state["number"]
    print(f"Node: process_odd - Processing odd number: {number}")
    new_action = f"Processed {number} as ODD."
    return {"action_taken": new_action}

In [4]:
# 3. Build the Graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("check_even_odd", check_even_odd)
workflow.add_node("process_even", process_even)
workflow.add_node("process_odd", process_odd)

# Set the entry point
workflow.set_entry_point("check_even_odd")

# Add edges (transitions)
# Conditional edge from check_even_odd
workflow.add_conditional_edges(
    "check_even_odd", # From this node
    lambda state: state["next_node"], # Based on this decision function
    { # Mapping of decisions to next nodes
        "process_even": "process_even",
        "process_odd": "process_odd",
    }
)

# Edges from processing nodes to END
workflow.add_edge("process_even", END)
workflow.add_edge("process_odd", END)

# Compile the graph
app = workflow.compile()

In [5]:
# 4. Run the Graph
print("--- Running with an EVEN number (4) ---")
input_state_even = {"number": 4, "action_taken": ""}
# For simplicity, we'll use a fixed thread_id. In a real app, this would be dynamic.
config_even = {"configurable": {"thread_id": "demo_even_run"}}
final_state_even = app.invoke(input_state_even, config=config_even)
print(f"Final State for even number: {final_state_even}")
print("-" * 30)

print("--- Running with an ODD number (7) ---")
input_state_odd = {"number": 7, "action_taken": ""}
config_odd = {"configurable": {"thread_id": "demo_odd_run"}}
final_state_odd = app.invoke(input_state_odd, config=config_odd)
print(f"Final State for odd number: {final_state_odd}")
print("-" * 30)

print("--- Running with another EVEN number (10) ---")
input_state_another_even = {"number": 10, "action_taken": ""}
config_another_even = {"configurable": {"thread_id": "demo_another_even_run"}}
final_state_another_even = app.invoke(input_state_another_even, config=config_another_even)
print(f"Final State for another even number: {final_state_another_even}")
print("-" * 30)

--- Running with an EVEN number (4) ---
Node: check_even_odd - Checking number: 4
Node: process_even - Processing even number: 4
Final State for even number: {'number': 4, 'action_taken': 'Processed 4 as EVEN.'}
------------------------------
--- Running with an ODD number (7) ---
Node: check_even_odd - Checking number: 7
Node: process_odd - Processing odd number: 7
Final State for odd number: {'number': 7, 'action_taken': 'Processed 7 as ODD.'}
------------------------------
--- Running with another EVEN number (10) ---
Node: check_even_odd - Checking number: 10
Node: process_even - Processing even number: 10
Final State for another even number: {'number': 10, 'action_taken': 'Processed 10 as EVEN.'}
------------------------------
